# ETL (Extracción, Transformación y Carga)
Hacer la  ETL de los datos es una parte fundamental en cualquier proyecto de análisis de datos.

## Importación de librerías necesarias


In [1]:
import pyodbc
import pandas as pd
import numpy as np
import os
import warnings  # Para ocultar warnings innecesarios

warnings.filterwarnings("ignore", category=UserWarning)

## Conectar las bases de datos